In [30]:
import pandas as pd

from gspread_pandas import Spread, Client

import src.mongos as mg
from src.common import *

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load

In [31]:
df_raw = mg.read_mongo('earthquake', 'geothermal')
df_raw.head()

,최대주입속도,최소주입속도,누적 배출,누적 주입,배출(PX-1),배출(PX-2),배출(합계),순 누적 주입,주입(PX-1),주입(PX-2),주입(합계),날짜,미소진동 관측 회수(규모 1 이상),미소진동 관측 회수(규모 2 이상)
0,8.00,0.17,-,23.04,,-,-,23.04,,23.04,23.04,1/29/2016,,
1,27.00,0.33,-,140.06,,-,-,140.06,,117.02,117.02,1/30/2016,,
2,46.83,0.33,-,365.86,,-,-,365.86,,225.80,225.80,1/31/2016,,
3,33.33,10.00,-,716.65,,-,-,716.65,,350.79,350.79,2/1/2016,,
4,25.00,1.83,-,919.29,,-,-,919.29,,202.64,202.64,2/2/2016,,


# Prepare

## Cols

In [32]:
eprint(df_raw.columns)
cols = [11, 10, 3, 2, 7, 12, 13]

0   최대주입속도
1   최소주입속도
2  누적 배출
3  누적 주입
4  배출(PX-1)
5  배출(PX-2)
6  배출(합계)
7  순 누적 주입
8  주입(PX-1)
9  주입(PX-2)
10  주입(합계)
11 날짜
12 미소진동 
관측 회수(규모 1 이상)
13 미소진동 
관측 회수(규모 2 이상)


## isna

In [33]:
df_raw.iloc[::, cols].isna()

,날짜,주입(합계),누적 주입,누적 배출,순 누적 주입,미소진동 관측 회수(규모 1 이상),미소진동 관측 회수(규모 2 이상)
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False


## Prepare

In [35]:
def get_date(x):
    # 1/29/2016
    import datetime
    format_ = '%m/%d/%Y'
    date = datetime.datetime.strptime(x, format_)
    return date


def get_numeric(x):
    try:
        result = float(x.strip().replace(',', ''))
        return result
    except:
        return 0


geo = df_raw.copy().iloc[::, cols]
geo['날짜'] = geo['날짜'].apply(get_date)
geo.iloc[::, [1, 2, 3, 4]] = geo.iloc[::, [1, 2, 3, 4]].applymap(get_numeric)

geo

,날짜,주입(합계),누적 주입,누적 배출,순 누적 주입,미소진동 관측 회수(규모 1 이상),미소진동 관측 회수(규모 2 이상)
0,2016-01-29,23.04,23.04,0.00,23.04,,
1,2016-01-30,117.02,140.06,0.00,140.06,,
2,2016-01-31,225.80,365.86,0.00,365.86,,
3,2016-02-01,350.79,716.65,0.00,716.65,,
4,2016-02-02,202.64,919.29,0.00,919.29,,
5,2016-02-03,162.34,1081.63,0.00,1081.63,,
6,2016-02-04,109.59,1191.22,0.00,1191.22,2,0
7,2016-02-05,98.95,1290.17,0.00,1290.17,,
8,2016-02-06,0.00,1290.17,0.00,1290.17,2,0
9,2016-02-07,0.00,1290.17,0.00,1290.17,3,0


# Export

In [36]:
dir_item = '/home/jake/Documents/News_Item/'
dir_proj = dir_item + 'Earthquake/'
dir_tableau = dir_proj + 'tableau_data/'

# geo.to_csv(dir_tableau + 'geothermal.tsv', sep='\t', index=False)